In [35]:
import pymysql

In [36]:
# 設定資料庫連線資訊
host = 'localhost'
port = 3306
user = 'root'
passwd = '******'
db = 'TESTDB'
charset = 'utf8mb4'

In [37]:
# 建立連線
conn = pymysql.connect(host=host, port=port, user=user, passwd=passwd, db=db, charset=charset)
print('Successfully connected!')

Successfully connected!


In [48]:
# 建立游標
cursor = conn.cursor()

In [49]:
# 試著 INSERT 一筆資料
# 注意 TABLE 欄位的資料型態

# 先寫好 SQL 語法
sql = """
INSERT INTO Staff (ID, Name, DeptId, Age, Gender, Salary, recordDt)
VALUES ('001', 'Mike', '002', 45, 'M', 60000, '2020-04-24 14:59:57');
"""

# 將指令放進 cursor 物件，並執行
cursor.execute(sql)

IntegrityError: (1062, "Duplicate entry '001' for key 'PRIMARY'")

In [40]:
# 也可以使用下列方式產生時間
import time
t = time.localtime()
dt = time.strftime('%Y-%m-%d %H:%M:%S', t)
# 試著再 INSERT 一筆資料
sql = """
INSERT INTO Staff (ID, Name, DeptId, Age, Gender, Salary, recordDt)
VALUES ('002', 'Judy', '002', 30, 'F', 48000, '{}');
""".format(dt)

# 將指令放進 cursor 物件，並執行
cursor.execute(sql)

IntegrityError: (1062, "Duplicate entry '002' for key 'PRIMARY'")

In [41]:
# 因 pymysql 預設不會自動 commit，故要記得 commit
conn.commit()

In [42]:
# 試著將資料表中的資料取出
# 先寫好 SQL 語法
sql = """
SELECT * FROM Staff;
"""

# 將指令放進 cursor 物件，並執行
cursor.execute(sql)

8

In [43]:
# 將查詢結果取出,宣告一個變數去接住它
data = cursor.fetchall()
data

(('001',
  'Mike',
  '002',
  45,
  'M',
  60000,
  datetime.datetime(2021, 2, 7, 15, 13, 45)),
 ('002',
  'Judy',
  '002',
  30,
  'F',
  48000,
  datetime.datetime(2021, 2, 7, 15, 13, 45)),
 ('003',
  'Allen',
  '001',
  22,
  'M',
  50000,
  datetime.datetime(2021, 2, 7, 15, 13, 45)),
 ('004',
  'Tom',
  '002',
  47,
  'M',
  47000,
  datetime.datetime(2021, 2, 7, 15, 13, 45)),
 ('005',
  'Jack',
  '003',
  36,
  'M',
  52000,
  datetime.datetime(2021, 2, 7, 15, 13, 45)),
 ('006',
  'Abby',
  '002',
  24,
  'F',
  45000,
  datetime.datetime(2021, 2, 7, 15, 13, 45)),
 ('007',
  'Trump',
  '001',
  80,
  'M',
  80000,
  datetime.datetime(2021, 2, 7, 15, 13, 45)),
 ('008',
  'Marry',
  '003',
  29,
  'F',
  87000,
  datetime.datetime(2021, 2, 7, 15, 13, 45)))

In [44]:
# 從資料庫將 datetime 形態的資料取出後，在 Python 中會變成 datetime 物件
# 將 datetime 物件轉為字串
data[1][6].strftime('%Y-%m-%d %H:%M:%S')

'2021-02-07 15:13:45'

In [45]:
sql = """
DELETE FROM Staff
"""
cursor.execute(sql)
conn.commit()

In [46]:
import json
import time

path = 'staff2.json'
with open(path, 'r', encoding='utf-8') as f:
    jsondata = json.loads(f.read())

jsondata

# 先寫好 SQL 語法
# 並將語法中會不斷改變的部分挖空 ( %s )
sql_insert = """
INSERT INTO Staff (ID, Name, DeptId, Age, Gender, Salary, recordDt)
VALUES (%s, %s, %s, %s, %s, %s, %s);
"""
# 整理 jsondata 成可 insert 進資料庫的樣子
import time
t = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime())
values = [tuple(jsondata[d].values()) + (t,) for d in jsondata]
values

# 將 SQL 批量執行
print('新增資料筆數:', cursor.executemany(sql_insert, values))

# Commit 並檢查資料是否存入資料庫
conn.commit()

新增資料筆數: 8


In [34]:
# 關閉游標及連線,先關閉游標再關閉連線(先建後關)
cursor.close()
conn.close()